In [6]:
from Preprocess import *
from Model import *
from FeatureEng import *

# Account
- account_id
- district_id
- date
- frequency

# Client
- client_id
- birth_number
- district_id

# Disposition
- disp_id
- client_id
- account_id
- type

# Loan
- loan_id
- account_id
- date
- amount
- duration
- payments
- status

# Transition
- trans_id
- account_id
- date
- type
- operation
- amount_balance
- k_symbol
- bank
- account

In [7]:
read_data()
make_dataframe(dic_csv)

